In [1]:
import os, sys
print("Current folder:", os.getcwd())
sys.path.insert(0, os.path.abspath(".."))
print("Project root added:", os.path.abspath(".."))

Current folder: /home/jovyan/EEP-153---Project-1/notebooks
Project root added: /home/jovyan/EEP-153---Project-1


In [2]:
from src.indicators import INDICATORS
from src.wb_fetch import fetch_wb_data
from src.panel import build_panel

In [3]:
import wbdata
import pandas as pd

countries_list = wbdata.get_countries()          # this is a LIST in your wbdata
countries_df = pd.DataFrame(countries_list)      # convert to DataFrame

countries_df.columns


Key '-3765247578634918696' not in persistent cache.
Key '-6781320151087764170' not in persistent cache.
Key '-246178064087034427' not in persistent cache.
Key '-2094931376535140325' not in persistent cache.
Key '-1198204470225275504' not in persistent cache.
Key '9087152426844380940' not in persistent cache.
Key '-4725164244312248060' not in persistent cache.
Key '-2045885935712134912' not in persistent cache.
Key '3905753198743471574' not in persistent cache.
Key '7170585300742029880' not in persistent cache.
Key '3970022354640234560' not in persistent cache.
Key '-8935553948502827766' not in persistent cache.
Key '-1726193178001173671' not in persistent cache.
Key '8559163719068482773' not in persistent cache.
Key '-2224390869658142409' not in persistent cache.
Key '5032203944981510217' not in persistent cache.
Key '-5751350643745114965' not in persistent cache.
Key '-3525901049626274156' not in persistent cache.
Key '-4036788251260401238' not in persistent cache.
Key '-7052860823713

Index(['id', 'iso2Code', 'name', 'region', 'adminregion', 'incomeLevel',
       'lendingType', 'capitalCity', 'longitude', 'latitude'],
      dtype='object')

In [4]:
panel = build_panel(df)
panel[["country", "year",
    "ag_export_share", "food_prod_index",
    "adult_mortality_male", "adult_mortality_female",
    "export_dependent"]].head()


NameError: name 'df' is not defined

In [ ]:
def region_value(x):
    if isinstance(x, dict):
        return x.get("value")
    return x
countries_df["region_value"] = countries_df["region"].apply(region_value)
countries_only = countries_df[countries_df["region_value"] != "Aggregates"].copy()
country_codes = countries_only["id"].tolist()
len(country_codes), countries_only[["id", "name", "region_value"]].head()

In [ ]:
df = fetch_wb_data(
    INDICATORS,
    start_year=2000,
    end_year=2022,
    cache_path="../data/wb_cache_all.csv"
)

panel = build_panel(df_countries)

panel["country"].nunique(), panel.head()


In [ ]:
df["country"].head(20)

In [ ]:
panel[["ag_export_share","food_prod_index","adult_mortality_male","adult_mortality_female"]].isna().mean().sort_values()

In [ ]:
import matplotlib.pyplot as plt

tmp = panel.dropna(subset=["adult_mortality_male", "ag_export_share"]).copy()
tmp["group"] = tmp["export_dependent"].map({True: "export-dependent", False: "not export-dependent"})

trend = tmp.groupby(["year", "group"])["adult_mortality_male"].mean().reset_index()

for g in trend["group"].unique():
    sub = trend[trend["group"] == g]
    plt.plot(sub["year"], sub["adult_mortality_male"], label=g)

plt.xlabel("Year")
plt.ylabel("Adult mortality (male, per 1,000)")
plt.legend()
plt.show()

In [ ]:
df_countries = fetch_wb_data(
    INDICATORS,
    start_year=2000,
    end_year=2022,
    countries=country_codes,   # <-- your filtered list (217 ISO3 codes)
    cache_path="../data/wb_cache_countries.csv")
df_countries.shape, df_countries["country"].nunique(), df_countries["country"].head(10)

In [ ]:
panel = build_panel(df_countries)
panel.shape, panel["country"].nunique()

In [ ]:
panel["country"].str.contains("Africa|World|income|states|dividend", case=False).mean()